In [1]:
# russian stopwords from https://github.com/mhq/train_punkt
# to be added: 
# emoticons

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
import re

In [2]:
header = ['text','label']

neg = pd.read_csv('negative.csv', delimiter = ';', names = header, usecols=[3,4])
pos = pd.read_csv('positive.csv', delimiter = ';', names = header, usecols=[3,4])

In [3]:
display(neg.head(5))
display(pos.head(5))

,text,label
0,на работе был полный пиддес :| и так каждое за...,-1
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1
2,@elina_4post как говорят обещаного три года жд...,-1
3,"Желаю хорошего полёта и удачной посадки,я буду...",-1
4,"Обновил за каким-то лешим surf, теперь не рабо...",-1


,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [4]:
neg['text'] = neg['text'].str.replace(r'[^а-я А-Я]', '')
pos['text'] = pos['text'].str.replace(r'[^а-я А-Я]', '')
display(neg.head(5))
display(pos.head(5))

,text,label
0,на работе был полный пиддес и так каждое закр...,-1
1,Коллеги сидят рубятся в а я изза долбанной в...,-1
2,как говорят обещаного три года ждут,-1
3,Желаю хорошего полта и удачной посадкия буду о...,-1
4,Обновил за какимто лешим теперь не работает п...,-1


,text,label
0,хоть я и школота но поверь у нас то же самое ...,1
1,Да всетаки он немного похож на него Но мой мал...,1
2,Ну ты идиотка я испугалась за тебя,1
3,Кто то в углу сидит и погибает от голода а м...,1
4,Вот что значит страшилка Но блинпосмотрев все...,1


In [5]:
df = pd.concat([neg, pos],ignore_index=True)
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./dataset_nlp.csv', index = False)
df = pd.read_csv('./dataset_nlp.csv')
df.head(10)

,text,label
0,ок я готова тебя выслушать Но к сожалению не ...,-1
1,за молочкомприспичило чтоль опчипокажи Хейли о,1
2,Доброе утро тви Через минут прибываю в Велик...,1
3,Целлофан мистер целлофан почему меня прет,1
4,Отлично покатались друг на другезато не холодноз,1
5,Завтра нужно пораньше в школу придти чтобы усп...,1
6,Все нормальные люди еще спят,1
7,Ну да теперь точно капец,1
8,никогда так сильно горло не болело даже говори...,-1
9,Хахахахахахахах раз два ТРИ зенит убирайтесь ...,1


In [6]:
# remove randomly rows to get subset of data if needed

remove_n = 200000
drop_indices = np.random.choice(df.index, remove_n, replace=False)
df_subset = df.drop(drop_indices)
print(len(df))
print(len(df_subset))

226834
26834


In [7]:
X_train = df.loc[:20000, 'text'].values
y_train = df.loc[:20000, 'label'].values
X_test = df.loc[6834:, 'text'].values
y_test = df.loc[6834:, 'label'].values

In [1]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#nltk.download()

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

In [3]:
# Processing documents into tokens
porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [6]:
# example of deleting stopwords
stop = stopwords.words('russian')
[w for w in tokenizer_porter('День был теплый, осенний и дождливый. Пространная перспектива, раскрывавшаяся с возвышения, где стояли русские батареи, защищавшие мост, то вдруг затягивалась кисейным занавесом косого дождя, то вдруг расширялась, и при свете солнца далеко и ясно становились видны предметы, точно покрытые лаком.')[-10:]
if w not in stop]

['солнца',
 'далеко',
 'ясно',
 'становились',
 'видны',
 'предметы,',
 'точно',
 'покрытые',
 'лаком.']

In [ ]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [ ]:
gs_lr_tfidf.fit(X_train, y_train)

In [ ]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)